In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9107088526354433228, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12662154510138727834
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14454161202677663126
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15876469556
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4230996228072969527
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
# from sklearn.metrics import log_loss
import sys
# import time
import math
import os
import pandas as pd
# from keras.callbacks import EarlyStopping
import numpy as np
# from glob import glob
# import cv2
# import skimage
# from skimage.transform import resize
from keras.utils.np_utils import to_categorical
# import keras
# from keras import layers
# from keras import models
# from keras import optimizers
# from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Input,concatenate
# from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
# from keras.applications.vgg19 import VGG19
from keras.regularizers import l2, l1

import matplotlib.pyplot as plt
# from keras.layers import Input, concatenate
# from keras import optimizers, metrics, models
# from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.3.1
tensorflow Version 2.0.0


In [5]:
batch_size = 64
img_height, img_width = 256, 256
input_shape = (img_height, img_width, 3)
epochs = 400

In [6]:
print(os.listdir("../input/"))

['payam-thesis', 'full-keras-pretrained-no-top']


In [7]:
# print(os.listdir("../input/full-keras-pretrained-no-top"))

In [8]:
train_dir = '../input/payam-thesis/train/'
test_dir = '../input/payam-thesis/test'

In [9]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [10]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect',
    validation_split= 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 4611 images belonging to 3 classes.
Found 1152 images belonging to 3 classes.
Found 765 images belonging to 3 classes.


In [11]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 4611
nb_validation_samples: 1152
nb_test_samples: 765

predict_size_train: 73
predict_size_validation: 18
predict_size_test: 12

 num_classes: 3


In [12]:
vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [13]:
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
from keras.applications import MobileNet

In [14]:
input_tensor = Input(shape = input_shape)  

base_model1=InceptionV3(input_shape= input_shape,weights=inception_weights, include_top=False, input_tensor=input_tensor)
base_model2=MobileNet(input_shape= input_shape,weights=mobilenet_weights, include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

merge = concatenate([x1, x2])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [15]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [16]:
c1 = model.layers[11].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[18].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[28].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[51].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[74].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[101].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[120].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[167].output
c8 = GlobalAveragePooling2D()(c8) 

c9 = model.layers[215].output
c9 = GlobalAveragePooling2D()(c9) 

c10 = model.layers[263].output
c10 = GlobalAveragePooling2D()(c10) 

c11 = model.layers[313].output
c11 = GlobalAveragePooling2D()(c11) 

c12 = model.layers[334].output
c12 = GlobalAveragePooling2D()(c12) 

c13 = model.layers[376].output
c13 = GlobalAveragePooling2D()(c13) 

c14 = model.layers[291].output
c14 = GlobalAveragePooling2D()(c14) 

c15 = model.layers[305].output
c15 = GlobalAveragePooling2D()(c15) 

c16 = model.layers[311].output
c16 = GlobalAveragePooling2D()(c16) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [17]:
os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "densenet201_InceptionResNetV2_descriptors"

In [18]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [19]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [20]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [21]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Machine Learning Algorithms

In [22]:
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score, classification_report

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import cross_val_score


k_fold = KFold(n_splits=10, shuffle=True, random_state=5)

In [23]:
y_test = np.asarray(test_labels)
y_test = np.argmax(y_test, axis=1)

y_train = np.asarray(train_labels)
y_train = np.argmax(y_train, axis=1)

# Bagging Classifier

In [24]:
clf = BaggingClassifier(RandomForestClassifier(), n_estimators = 300 )
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('AdaBoost Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 1.0
Test accuracy 0.9934640522875817
AdaBoost Classifier test accuracies 0.9935
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       329
           1       1.00      0.98      0.99       211
           2       1.00      1.00      1.00       225

    accuracy                           0.99       765
   macro avg       1.00      0.99      0.99       765
weighted avg       0.99      0.99      0.99       765

0.9934640522875817


In [25]:
clf = BaggingClassifier(RandomForestClassifier(), n_estimators = 300 )
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("BaggingClassifier - Test Accuracy on all folds: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 96.3483 and (STDEV 0.0283)
Best result for fold 2
Best accuracy is 0.987012987012987
Scores of all folds: [0.8961039  0.96103896 0.98701299 0.93506494 0.97402597 0.97368421
 0.94736842 0.98684211 0.98684211 0.98684211]
BaggingClassifier - Test Accuracy on all folds: 0.96 (+/- 0.06)


# AdaBoost Classifier

In [26]:
clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators = 300 )
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('AdaBoost Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 1.0
Test accuracy 0.9372549019607843
AdaBoost Classifier test accuracies 0.9373
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       329
           1       0.89      0.92      0.90       211
           2       0.98      0.95      0.96       225

    accuracy                           0.94       765
   macro avg       0.94      0.94      0.94       765
weighted avg       0.94      0.94      0.94       765

0.9372549019607843


In [27]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("(AdaBoost Classifier) Test Accuracy on all folds: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 85.3674 and (STDEV 0.0332)
Best result for fold 6
Best accuracy is 0.8947368421052632
Scores of all folds: [0.81818182 0.81818182 0.85714286 0.85714286 0.85714286 0.78947368
 0.89473684 0.89473684 0.86842105 0.88157895]
(AdaBoost Classifier) Test Accuracy on all folds: 0.85 (+/- 0.07)


# Decision Tree Classifier

In [28]:
clf = DecisionTreeClassifier()
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('DecisionTree Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 1.0
Test accuracy 0.9437908496732026
DecisionTree Classifier test accuracies 0.9438
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       329
           1       0.91      0.92      0.92       211
           2       0.98      0.94      0.96       225

    accuracy                           0.94       765
   macro avg       0.94      0.94      0.94       765
weighted avg       0.94      0.94      0.94       765

0.9437908496732026


In [29]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("DecisionTree - Test Accuracy on all folds: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 85.5058 and (STDEV 0.0356)
Best result for fold 9
Best accuracy is 0.9210526315789473
Scores of all folds: [0.80519481 0.79220779 0.85714286 0.85714286 0.84415584 0.85526316
 0.89473684 0.86842105 0.85526316 0.92105263]
DecisionTree - Test Accuracy on all folds: 0.86 (+/- 0.07)


# Random Forest Classifier

In [30]:
clf = RandomForestClassifier(n_estimators=5)
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('RandomForest Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 0.9971806549555411
Test accuracy 0.9699346405228758
RandomForest Classifier test accuracies 0.9699
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       329
           1       0.95      0.96      0.96       211
           2       0.99      0.96      0.98       225

    accuracy                           0.97       765
   macro avg       0.97      0.97      0.97       765
weighted avg       0.97      0.97      0.97       765

0.9699346405228758


In [31]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, test_labels, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("(RandomForest) Test Accuracy on all folds: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 88.4877 and (STDEV 0.0496)
Best result for fold 7
Best accuracy is 0.9605263157894737
Scores of all folds: [0.88311688 0.84415584 0.92207792 0.92207792 0.92207792 0.82894737
 0.80263158 0.96052632 0.84210526 0.92105263]
(RandomForest) Test Accuracy on all folds: 0.88 (+/- 0.10)


# XGBoost Classifier

In [32]:
clf = xgb.XGBClassifier()
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('XGB Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 0.9995662546085448
Test accuracy 0.9973856209150327
XGB Classifier test accuracies 0.9974
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       329
           1       1.00      0.99      1.00       211
           2       1.00      1.00      1.00       225

    accuracy                           1.00       765
   macro avg       1.00      1.00      1.00       765
weighted avg       1.00      1.00      1.00       765

0.9973856209150327


In [33]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("XGB - Test Accuracy on all folds: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 97.2573 and (STDEV 0.0169)
Best result for fold 7
Best accuracy is 1.0
Scores of all folds: [0.93506494 0.97402597 0.97402597 0.97402597 0.98701299 0.96052632
 0.96052632 1.         0.98684211 0.97368421]
XGB - Test Accuracy on all folds: 0.97 (+/- 0.03)


In [ ]:
print("Done!")